6. Провести анализ настроений на основе библиотеки Dostoevsky (модель была обучена на наборе данных RuSentiment) из любого русскоязычного произведения.

In [2]:
!pip install dostoevsky

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393380 sha256=62caac754af422870fb506cce0f2de3e001796fbfbf0d6daca8a78004b73246e
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [5]:
!python3 -m dostoevsky download fasttext-social-network-model

In [7]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [9]:
messages = [
    'О боже! как это все отвратительно! И неужели, неужели я… нет, это вздор, \
    это нелепость! И неужели такой ужас мог прийти \
    мне в голову? На какую грязь способно, однако, мое сердце!',
    'Приятно слышать, что вы так вежливо обращаетесь с котом.',
    'Всегда неприятно видеть, что человек, которого ты считаешь хуже и ниже \
    себя, любит или ненавидит то же, что и ты, и, таким образом, становится \
    похож на тебя.'
]

results = model.predict(messages, k=3)

for message, sentiment in zip(messages, results):
    print(message, '->', sentiment)

О боже! как это все отвратительно! И неужели, неужели я… нет, это вздор,     это нелепость! И неужели такой ужас мог прийти     мне в голову? На какую грязь способно, однако, мое сердце! -> {'negative': 0.7057950496673584, 'positive': 0.41490885615348816, 'skip': 0.16027602553367615}
Приятно слышать, что вы так вежливо обращаетесь с котом. -> {'neutral': 0.5621865391731262, 'positive': 0.29422497749328613, 'skip': 0.040855713188648224}
Всегда неприятно видеть, что человек, которого ты считаешь хуже и ниже     себя, любит или ненавидит то же, что и ты, и, таким образом, становится     похож на тебя. -> {'negative': 0.40734341740608215, 'positive': 0.14034625887870789, 'neutral': 0.11597072333097458}


7. Произвести ансамблирование не менее 2 моделей любой нейронной сети, рассказать насколько произошло улучшение выходного параметра.

In [42]:
import pandas as pd
from sklearn import datasets

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

In [43]:
df = pd.read_csv("Iris.csv")
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [47]:
dataset = df.values

train = dataset[:,0:5].astype(float)
target = dataset[:,5]

encoder = LabelEncoder()
encoder.fit(target)
target = encoder.transform(target)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(train, target, 
                                                    test_size=0.15)

In [51]:
mdl1 = LogisticRegression(penalty='l2', solver='liblinear', random_state=5)
mdl1 = mdl1.fit(X_train, y_train)
y_pred = mdl1.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       0.67      0.67      0.67         3
           2       0.92      0.92      0.92        12

    accuracy                           0.91        23
   macro avg       0.86      0.86      0.86        23
weighted avg       0.91      0.91      0.91        23



In [54]:
mdl2 = RandomForestClassifier(criterion="entropy", max_depth=5, 
                              min_samples_split=10, 
                              min_samples_leaf=5, random_state=5)
mdl2 = mdl2.fit(X_train, y_train)
y_pred = mdl2.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00        12

    accuracy                           1.00        23
   macro avg       1.00      1.00      1.00        23
weighted avg       1.00      1.00      1.00        23



In [55]:
model_1 = LogisticRegression(penalty='l2', solver='liblinear', random_state=5)
model_2 = RandomForestClassifier(criterion="entropy", max_depth=5, 
                                 min_samples_split=20, 
                                 min_samples_leaf=10, random_state=5)
 
final_model = VotingClassifier(
    estimators=[('lr', model_1), ('rf', model_2)], voting='soft')
 
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00        12

    accuracy                           1.00        23
   macro avg       1.00      1.00      1.00        23
weighted avg       1.00      1.00      1.00        23

